In [ ]:
from sentence_transformers import SentenceTransformer
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ast
import plotly

from dotenv import load_dotenv
import os
import requests

## Variables de entorno

In [49]:
# Cargar variables de entorno
load_dotenv()

# Leer la API Key
api_key = os.getenv("GROQ_API_KEY")


## Funciones

In [7]:
def safe_parse_conocimientos(value):
    '''
    Cambia de formato las entradas de la forma
    "'palabra_1','palabra_2',..., 'palabra_n'"
    a
    ['palabra_1','palabra_2,...,'palabra_n']
    '''
    try:
        # Si no está entre corchetes, lo envolvemos
        if not value.strip().startswith("["):
            value = f"[{value}]"
        return ast.literal_eval(value)
    except Exception:
        return [str(value)]  # Si no se puede evaluar, lo devolvemos como lista con un solo string

## Lectura datos

In [8]:
conoc_df=pd.read_excel("../data/conoc_tej.xlsx",index_col=False).copy()
conoc_df=conoc_df.replace('NA',np.nan)
conoc_df.shape

(3087, 26)

In [9]:
#Conocimientos de la vacante
conoc_df['conocimientos'][0]

"'mantenimiento de maquinaria industrial', 'manejo de bombas de vacío', 'equipos de soldadura', 'mecánica hidráulica'"

## Clusterizarion semántica

In [11]:
#1. Define the corpus
corpus = conoc_df['conocimientos'].apply(safe_parse_conocimientos).sum()

In [13]:
# 2. Embed skills
model = SentenceTransformer('all-MiniLM-L6-v2')
skill_embeddings = model.encode(corpus)

print(skill_embeddings.shape)

(16508, 384)


In [14]:
# 3. Compute hierarchical clustering
linked = linkage(skill_embeddings, method='ward')  # "ward" tries to minimize variance

In [5]:
import plotly.figure_factory as ff
# 4. Create Plotly Dendrogram
fig = ff.create_dendrogram(skill_embeddings, labels=corpus, linkagefun=lambda x: linkage(x, 'ward'))

fig.update_layout(
    width=1200,
    height=800,
    title="Dynamic Skill Clustering Dendrogram",
    xaxis_title="Skills",
    yaxis_title="Distance",
    showlegend=False
)

fig.write_html("../output/skill_dendogram.html")

In [17]:
# 5. Optional: Cut the dendrogram into "k" clusters
num_clusters = 10
cluster_labels = fcluster(linked, num_clusters, criterion='maxclust')

In [24]:
# 6. Print results
n=1
for skill, cluster_id in zip(corpus, cluster_labels):
    if n<=10:
        print(f"Skill: {skill}, Cluster: {cluster_id}")
        n+=1
    else: 
        break

Skill: mantenimiento de maquinaria industrial, Cluster: 7
Skill: manejo de bombas de vacío, Cluster: 7
Skill: equipos de soldadura, Cluster: 7
Skill: mecánica hidráulica, Cluster: 7
Skill: Control integrado de plagas, Cluster: 6
Skill: Control de roedores, Cluster: 6
Skill: Desinsectación, Cluster: 7
Skill: Lavado y desinfección de tanques, Cluster: 7
Skill: Curso de trabajo en alturas, Cluster: 5
Skill: Licencias A2 y C1, Cluster: 7


In [ ]:
#7. Label the clusters using an LLM:

# For each cluster, summarize its skills
from collections import defaultdict

cluster_skills = defaultdict(list)
for skill, cluster_id in zip(corpus, cluster_labels):
    cluster_skills[cluster_id].append(skill)

for cluster_id, cluster_list in cluster_skills.items():
    print(f"Cluster {cluster_id}: {cluster_list}")
    # You can send cluster_list to GPT to "summarize" it into a label

Cluster 7: ['mantenimiento de maquinaria industrial', 'manejo de bombas de vacío', 'equipos de soldadura', 'mecánica hidráulica', 'Desinsectación', 'Lavado y desinfección de tanques', 'Licencias A2 y C1', 'Inglés', 'Gestora y consultora asesora', 'NO_APLICA', 'Mercado de energía colombiano', 'Cobro de cartera', 'Manejo de tecnicos', 'Medida directa y semidirecta', 'Construcción', 'Manejo de correspondencia', 'Regulaciones gubernamentales', 'Herramienta SAP', 'Planeación de compras y contratación', 'Herramienta SAP', 'NO_APLICA', 'NO_APLICA', 'Equipos de calentamiento', 'Sistema de aire a presión', 'Consumo de agua', 'Huella de carbono', 'Mejora continua', 'Limpieza y lubricación de máquinas y equipos', 'Mantenimientos preventivos de maquinaria', 'Lavados de techos de casetas', 'Pintar', 'Sondeo de tubería de aguas negras', 'Pegar pisos cerámicos', 'Instalación de llaves de sensor', 'Trabajos de soldadura', 'Control de aguas', 'Control de aguas', 'Lubricación', 'Uso de herramientas', 'D

## Etiquetar el cluster con un LLM

In [53]:
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

#E.g Primer cluster
skills_list = cluster_skills[2][:500]

# Spanish prompt
prompt = f"Dado el siguiente listado de habilidades/conocimientos laborales: {', '.join(skills_list)}, sugiere un nombre de max 10 palabras que resuma las habilidades"

payload = {
    "model": "llama3-8b-8192",  # Or "llama3-70b-8192"
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.3
}

response = requests.post("https://api.groq.com/openai/v1/chat/completions", json=payload, headers=headers)

# Verificamos si todo salió bien
if response.status_code == 200:
    data = response.json()
    print(data["choices"][0]["message"]["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

Basado en el listado de habilidades y conocimientos laborales, te sugiero el siguiente nombre:

**"Electromecánico de Alta Tensión"**

Este nombre resume las habilidades en electrónica, mecanica, automatización, diseño, instalación, mantenimiento y supervisión de redes eléctricas, motores eléctricos, sistemas de control y protección, así como la experiencia en proyectos de seguridad electrónica, hidroelectricas, subestaciones eléctricas y construcción de líneas de alta tensión.

Si deseas un nombre más breve, también podrías considerar:

**"Electromecánico"**
**"Tecnico en Redes Eléctricas"**
**"Ingeniero Electricista"**
**"Tecnico en Energía Eléctrica"**

Recuerda que el nombre debe ser claro y conciso, y reflejar las habilidades y experiencia laboral del profesional.
